In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

In [4]:
items_urls = []

In [8]:
for i in tqdm(range(1, 272)):
    if i == 1:
        end = '/'
    else:
        end = f'&page={i}'
    link = f'https://kino.mail.ru/series/all/?order=rate_count&year=1999&year=2024{end}'
    content = requests.get(link).content
    soup = BeautifulSoup(content, 'lxml')
    series = soup.find_all(class_='link link_inline link-holder link-holder_itemevent link-holder_itemevent_small')

    for item in series:
        item_page_url = item.get('href')
        items_urls.append('https://kino.mail.ru' + item_page_url)

100%|██████████| 271/271 [03:44<00:00,  1.21it/s]


In [9]:
urls = pd.DataFrame({'url':items_urls})
urls.to_csv('urls_mail_ru.csv')

items_urls = items_urls[:10000]

len(items_urls)

10000

In [10]:
items_urls[0]

'https://kino.mail.ru/series_756333_velikolepnii_vek/'

In [11]:
l1, l2, l3, l4 = items_urls[:2500], items_urls[2500:5000], items_urls[5000:7500], items_urls[7500:10000]

In [12]:
l5 = items_urls[2000:2500]

In [23]:
titles = []
sources = []
descriptions = []
ganres = []
years = []
posters = []
casts = []
ratings = []
s = 1

for url in tqdm(l2):
    try:
        text = requests.get(url).text
        soup = BeautifulSoup(text, 'lxml')
        try:
            title = soup.find('h1', class_='text text_bold_giant color_white').text
        
            titles.append(title[:title.find(' (сери')])
        except:
            print('маслина')
            titles.append(None)
        # тянем постер
        try:
            picture_url = soup.find('meta', itemprop='image')['content']
            posters.append(picture_url)
        except:
            posters.append(None)
        # тянем жанры
        try:
            ganre = soup.find_all('span', class_='badge__text')
            helper = []
            for i in ganre:
                helper.append(i.text)
            ganres.append(helper)
        except:
            ganres.append(None)
        # тянем cast
        try:
            cast = soup.find_all('span', class_='p-truncate__inner js-toggle__truncate-inner')
            a = []
            for act in cast:
                a.append(act.text)
            casts.append(a[1:4])
        except:
            casts.append(None)
        try:
            description = soup.find('div', class_='text text_inline text_light_medium text_fixed valign_baseline p-movie-info__description-text').text
            descriptions.append(description.replace('\xa0', ' '))
        except:
            descriptions.append(None)
        # год
        try:
            year = soup.find('span', class_='nowrap').text
            years.append(year.split()[:3])
        except:
            years.append(None)
        # imdb
        try:
            rating = float(soup.find('span', class_='text text_bold_huge text_fixed').text)
            ratings.append(rating)
        except:
            ratings.append(None)

        sources.append(url)

        if (len(sources) % 500 == 0) or (l2[-1] == url):
            data = pd.DataFrame({'url':sources,
                                'poster':posters,
                                'title':titles,
                                'ganres':ganres,
                                'description':descriptions,
                                'year':years,
                                'rating':ratings,
                                'cast':casts})
            print(f'{len(data)} saved')

            name = f'l2/data_{s}.csv'

            data.to_csv(name)

            s += 1
            titles = []
            sources = []
            descriptions = []
            ganres = []
            years = []
            posters = []
            casts = []
            ratings = []

    except requests.exceptions.RequestException as e:
        print(f"ссылка номер: {items_urls.index(url)}, ссылка: {url}, ошибка: {e}")

    

  3%|▎         | 77/2500 [02:37<14:47:09, 21.97s/it]

ссылка номер: 2576, ссылка: https://kino.mail.ru/series_769682_virtuozi/, ошибка: HTTPSConnectionPool(host='kino.mail.ru', port=443): Max retries exceeded with url: /series_769682_virtuozi/ (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1006)')))


 20%|██        | 501/2500 [07:57<24:29,  1.36it/s]  

500 saved


 40%|████      | 1001/2500 [14:09<17:40,  1.41it/s] 

...

500 saved


 80%|████████  | 2001/2500 [25:51<04:57,  1.68it/s]

500 saved


100%|██████████| 2500/2500 [31:39<00:00,  1.32it/s]

499 saved


In [15]:
len(titles)

499

In [24]:
indx = [2576]
urls_false = []
for ind in indx:
    urls_false.append(items_urls[ind])

In [25]:
def info_get(urls: str):

    titles = []
    sources = []
    descriptions = []
    ganres = []
    years = []
    posters = []
    casts = []
    ratings = []

    for url in tqdm(urls):
        try:
            text = requests.get(url).text
            soup = BeautifulSoup(text, "lxml")
            try:
                title = soup.find("h1", class_="text text_bold_giant color_white").text

                titles.append(title[: title.find(" (сери")])
            except:
                print("маслина")
                titles.append(None)
            # тянем постер
            try:
                picture_url = soup.find("meta", itemprop="image")["content"]
                posters.append(picture_url)
            except:
                posters.append(None)
            # тянем жанры
            try:
                ganre = soup.find_all("span", class_="badge__text")
                helper = []
                for i in ganre:
                    helper.append(i.text)
                ganres.append(helper)
            except:
                ganres.append(None)
            # тянем cast
            try:
                cast = soup.find_all(
                    "span", class_="p-truncate__inner js-toggle__truncate-inner"
                )
                a = []
                for act in cast:
                    a.append(act.text)
                casts.append(a[1:4])
            except:
                casts.append(None)
            # тянем описание
            try:
                description = soup.find(
                    "div",
                    class_="text text_inline text_light_medium text_fixed valign_baseline p-movie-info__description-text",
                ).text
                descriptions.append(description.replace("\xa0", " "))
            except:
                descriptions.append(None)
            # год
            try:
                year = soup.find("span", class_="nowrap").text
                years.append(year.split()[:3])
            except:
                years.append(None)
            # imdb
            try:
                rating = float(
                    soup.find("span", class_="text text_bold_huge text_fixed").text
                )
                ratings.append(rating)
            except:
                ratings.append(None)

            sources.append(url)

            if len(sources) == len(urls):
                data = pd.DataFrame(
                    {
                        "url": sources,
                        "poster": posters,
                        "title": titles,
                        "ganres": ganres,
                        "description": descriptions,
                        "year": years,
                        "rating": ratings,
                        "cast": casts,
                    }
                )

                print(f"{len(data)} saved")
                name = f"l2/data_{666}.csv"

                data.to_csv(name)

                titles = []
                sources = []
                descriptions = []
                ganres = []
                years = []
                posters = []
                casts = []
                ratings = []

        except requests.exceptions.RequestException as e:
            print(f"ссылка номер: {items_urls.index(url)}, ссылка: {url}, ошибка: {e}")


info_get(urls_false)

100%|██████████| 1/1 [00:00<00:00,  1.31it/s]

1 saved


In [17]:
data = pd.DataFrame({'url':sources,
                                'poster':posters,
                                'title':titles,
                                'ganres':ganres,
                                'description':descriptions,
                                'year':years,
                                'rating':ratings,
                                'cast':casts})
print(f'{len(data)} saved')
name = f'l1/data_{5}.csv'

data.to_csv(name)

499 saved


In [13]:
l1[2500]

IndexError: list index out of range

In [8]:
len(l1)

2500

In [29]:
import os

directory = "l1_l2"  # 'l4'

for i, filename in enumerate(os.listdir(directory)):
    path = os.path.join(directory, filename)
    if i == 0:
        df = pd.read_csv(path).drop("Unnamed: 0", axis=1)
    else:
        df2 = pd.read_csv(path).drop("Unnamed: 0", axis=1)
        df = pd.concat([df, df2])

df.to_csv("l1_l2/first5000.csv")

In [34]:
df = pd.read_csv("l1_l2/first5000.csv")
df.drop_duplicates(subset=["url"]).info()

<class 'pandas.core.frame.DataFrame'>
Index: 5000 entries, 0 to 5001
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   5000 non-null   int64  
 1   url          5000 non-null   object 
 2   poster       5000 non-null   object 
 3   title        5000 non-null   object 
 4   ganres       5000 non-null   object 
 5   description  5000 non-null   object 
 6   year         5000 non-null   object 
 7   rating       4017 non-null   float64
 8   cast         5000 non-null   object 
dtypes: float64(1), int64(1), object(7)
memory usage: 390.6+ KB


In [35]:
df.to_csv("l1_l2/first5000drop.csv")

In [36]:
# Определите функцию для извлечения года
def extract_first_year(year_string):
    # Удаление символов квадратных скобок
    year_string = year_string.strip("[]")
    # Разделение строки по запятой и извлечение первого элемента
    first_year = year_string.split(",")[0].strip(" '\"")
    return first_year

In [37]:
df = pd.read_csv("data_mail.csv")

In [38]:
# Примените функцию ко всему столбцу 'year'
df["year1"] = df["year"].apply(extract_first_year)

In [41]:
import ast

# Функция для преобразования строки в желаемый формат
def format_genres(genre_string):
    # Оцените строку как список, если она не в списковом формате
    if isinstance(genre_string, str):
        genres_list = ast.literal_eval(genre_string)
    else:
        genres_list = genre_string  # Если это уже список

    # Преобразуйте каждый элемент списка в строку с заглавной буквы
    genres_list = [genre.capitalize() for genre in genres_list]

    # Объедините элементы списка в одну строку через запятую
    return ", ".join(genres_list)

In [42]:
# Примените функцию ко всему столбцу 'genres'
df["genres"] = df["ganres"].apply(format_genres)

In [43]:
df

,url,poster,title,ganres,description,year,rating,cast,year1,genres
0,https://kino.mail.ru/series_776858_opera_hroni...,https://resizer.mail.ru/p/9667363a-0ebb-5334-8...,Опера: Хроники убойного отдела,"['боевик', 'криминал', 'детектив']","Абдулова, Ларин, Дукалис и Мухомор — главные г...","['2004', '—', '2006']",6.5,"['Владимир Шевельков, Юрий Кузнецов, Анастасия...",2004,"Боевик, Криминал, Детектив"
1,https://kino.mail.ru/series_932003_pingvini_mo...,https://resizer.mail.ru/p/355bff9e-6259-5c97-9...,Пингвины моей мамы,"['драма', 'комедия']",Гоша — обычный 15-летний московский школьник. ...,"['2021', '(Сериал', 'закончился)']",7.1,"['Макар Хлебников, Александра Урсуляк, Алексей...",2021,"Драма, Комедия"
2,https://kino.mail.ru/series_811036_silnee_sudbi/,https://resizer.mail.ru/p/529905e7-f544-5640-a...,Сильнее судьбы,['мелодрама'],Героиня сериала «Сильнее судьбы» Аня Колчина —...,"['2014', '(Сериал', 'закончился)']",5.8,"['Наталия Антонова, Анастасия Ричи, Любовь Тол...",2014,Мелодрама
3,https://kino.mail.ru/series_795532_vtorzhenie_...,https://resizer.mail.ru/p/e6e44d1a-5a23-58c3-b...,Атака титанов,"['драма', 'фэнтези', 'боевик', 'аниме', 'мульт...","Гиганты это огромные существа, ростом с многоэ...","['2013', '—', '2023']",7.6,"['Юки Кадзи, Марина Иноуэ, Юи Исикава, Кисё Та...",2013,"Драма, Фэнтези, Боевик, Аниме, Мультсериалы"
4,https://kino.mail.ru/series_902636_maiya/,https://resizer.mail.ru/p/5e902e02-ca71-5661-8...,Майя,['детектив'],Год назад 15-летняя Майя пережила страшную тра...,"['2016', '(Сериал', 'закончился)']",7.1,"['Мария Белоненко, Елена Оболенская, Александр...",2016,Детектив
...,...,...,...,...,...,...,...,...,...,...
9994,https://kino.mail.ru/series_917662_grezi/,https://resizer.mail.ru/p/c2285ef7-bb15-5710-b...,Грезы,"['драма', 'триллер']",Главная героиня сериала «Грезы» — бывший детек...,"['2018', '(Сериал', 'закончился)']",NaN,"['Сара Шахи, Деннис Хейсберт, Кэтрин Моррис, Д...",2018,"Драма, Триллер"
9995,https://kino.mail.ru/series_933510_dvadtsat_py...,https://resizer.mail.ru/p/4a1a2425-c648-52ef-a...,Двадцать пять и двадцать один,"['драма', 'мелодрама', 'спорт']",Действие сериала разворачивается в Южной Корее...,"['2022', '(Сериал', 'закончился)']",NaN,"['Ким Тхэ-ри, Нам Джу-хек, Пона, Чхве Хен-ук, ...",2022,"Драма, Мелодрама, Спорт"
9996,https://kino.mail.ru/series_922104_dve_materi/,https://resizer.mail.ru/p/c43b775b-17f3-52ea-a...,Две матери,"['драма', 'мелодрама']","Неожиданная смерть Сергея, погибшего в автокат...","['2018', '(Сериал', 'закончился)']",NaN,"['Ольга Атанасова, Зоряна Марченко, Михаил Пше...",2018,"Драма, Мелодрама"
9997,https://kino.mail.ru/series_918539_dobro_pozha...,https://resizer.mail.ru/p/7dbf4fcf-9ff4-59ed-a...,Добро пожаловать в бальный зал,"['аниме', 'музыкальные']",Татара — ученик средней школы и типичный интро...,"['2017', '(Сериал', 'закончился)']",NaN,"['Симба Цутия, Аянэ Сакура, Нобухико Окамото, ...",2017,"Аниме, Музыкальные"
